In [1]:
import os
import skimage 
import numpy as np
from tqdm import tqdm
from tqdm.contrib import itertools as it

from pylibCZIrw import czi as pyczi
from cztile.fixed_total_area_strategy import AlmostEqualBorderFixedTotalAreaStrategy2D


import matplotlib.pyplot as plt
# from scroutines.config_plots import *

/home/qlyu/mambaforge/envs/czi2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

ddir = '/scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs'
f_r3v1 = os.path.join(ddir, 'LHA3_R3_tiny_V01.czi') 
f_r3v2 = os.path.join(ddir, 'LHA3_R3_tiny_V02.czi') 
f_r5v1 = os.path.join(ddir, 'LHA3_R5_tiny_V01.czi') 
f_r5v2 = os.path.join(ddir, 'LHA3_R5_tiny_V02.czi') 

f_r3m  = os.path.join(ddir, 'LHA3_R3_tiny.mvl') 
f_r5m  = os.path.join(ddir, 'LHA3_R5_tiny.mvl') 


In [3]:
f_all = [
    os.path.join(ddir, 'LHA3_R3_tiny_V01.czi'),
    os.path.join(ddir, 'LHA3_R3_tiny_V02.czi'), 
    os.path.join(ddir, 'LHA3_R5_tiny_V01.czi'),
    os.path.join(ddir, 'LHA3_R5_tiny_V02.czi'),
]

In [4]:
def tile_one_file(f_src, f_dst_all=[]):
    # create a "tile" 
    tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(
        total_tile_width=350,
        total_tile_height=500,
        min_border_width=10,
    )

    # create CZI instance to read some metadata 
    with pyczi.open_czi(f_src) as czidoc_r:

        # get the size of the bounding rectange for the scence
        print(czidoc_r.total_bounding_box)
        print(czidoc_r.total_bounding_rectangle)
        print(czidoc_r.scenes_bounding_rectangle)
        bbox = czidoc_r.total_bounding_box
        numZ, numC = bbox['Z'][1], bbox['C'][1]
        tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[0])

    # show the created tile locations
    for i, tile in enumerate(tiles):
        print(tile.roi)
        outdir = os.path.join(ddir, f't{i}')
        if not os.path.isdir(outdir):
            os.mkdir(outdir)
        f_dst = os.path.join(outdir, os.path.basename(f_src)) #.replace('.czi', f'_t{i}.czi'))
        f_dst_all.append(f_dst)

    with pyczi.open_czi(f_src) as czidoc_r:
        # loop over all tiles 
        for i, tile in enumerate(tiles):
            f_dst = f_dst_all[i]
            print(f"tile {i}, {f_dst}")
            with pyczi.create_czi(f_dst, exist_ok=True) as czidoc_w:
                for z, c in it.product(range(numZ), range(numC)):
                    plane = {'Z': z, 'C': c}

                    # read and write plane by plane for the same tile
                    tile2d = czidoc_r.read(
                        plane=plane,
                        roi=(
                            tile.roi.x,
                            tile.roi.y,
                            tile.roi.w,
                            tile.roi.h
                        )
                    )
                    czidoc_w.write(
                        tile2d,
                        plane=plane,
                        location=(
                            tile.roi.x,
                            tile.roi.y
                        )
                    )

In [5]:
for f_src in f_all:
    print(f".....processing: {f_src}.............")        
    tile_one_file(f_src, f_dst_all=[])

.....processing: /scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/LHA3_R3_tiny_V01.czi.............
{'T': (0, 1), 'Z': (0, 151), 'C': (0, 2), 'R': (0, 1), 'I': (0, 1), 'H': (0, 1), 'V': (0, 1), 'X': (0, 660), 'Y': (0, 920)}
Rectangle(x=0, y=0, w=660, h=920)
{0: Rectangle(x=0, y=0, w=660, h=920)}
Rectangle(x=0, y=0, w=350, h=500)
Rectangle(x=0, y=420, w=350, h=500)
Rectangle(x=310, y=0, w=350, h=500)
Rectangle(x=310, y=420, w=350, h=500)
tile 0, /scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/t0/LHA3_R3_tiny_V01.czi


100%|██████████| 302/302 [00:00<00:00, 622.51it/s]


tile 1, /scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/t1/LHA3_R3_tiny_V01.czi


100%|██████████| 302/302 [00:00<00:00, 618.72it/s]


tile 2, /scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/t2/LHA3_R3_tiny_V01.czi


...

100%|██████████| 302/302 [00:00<00:00, 623.09it/s]


tile 2, /scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/t2/LHA3_R5_tiny_V02.czi


100%|██████████| 302/302 [00:00<00:00, 605.12it/s]


tile 3, /scr/qlyu/projects/v1/data/multifish_demo_tiny/inputs/t3/LHA3_R5_tiny_V02.czi


100%|██████████| 302/302 [00:00<00:00, 593.92it/s]
